In [ ]:
#r "nuget:TorchSharp-cpu"
open TorchSharp
open type TorchSharp.TensorExtensionMethods

# Tensors

In TorchSharp, as in all deep learning, the fundamental data type is a 'tensor,' which is simply a generalized matrix. In Linear Algebra, a one-dimensional arrays are called 'vector,' and a two-dimensional array is a 'matrix.' Generalizing on that, a tensor is simply an N-dimensional array. 

Please note that there is an overloaded use of the word 'dimensions' here -- in physics, a vector (one dimension) with three elements is used to represents a point in space, one element for each spatial dimension. When we speak of 'dimension' in these tutorials, it is the number of tensor dimensions that is of interest.

So, let's get started with tensors by creating some.

## Constant-Filled Tensors

The simplest tensor creation primitives just initializes a tensor with either 0 or 1 in all its elements. The arguments passed in is the size of each dimension. Think of the first dimension as the rows of a table, the second as the columns, and then you just have to generalize things in your head after that. In the examples below, we'll mostly be creating 3x4 matrices, for simplicity.

One thing to note is that .NET Interactive will show the object and its fields, etc. when you say just 't' at the end of the notebook cell. What we want for tensors is to show the contents, and there's a special version of ToString() taking a Boolean that shows not just the size and type of the tensor, but also its contents.

In [ ]:
let t = torch.ones(3L, 4L)
t.ToString(true)